In [1]:
# This notebook assumes to be running from your FireCARES VM (eg. python manage.py shell_plus --notebook --no-browser)

import sys
import os
import time
import pandas as pd
import numpy as np
sys.path.insert(0, os.path.realpath('..'))
import folium
import django
django.setup()
from django.db import connections
from pretty import pprint
from firecares.firestation.models import (FireDepartment, FireStation, NFIRSStatistic, FireDepartmentRiskModels,
                                          PopulationClassQuartile)
from fire_risk.models import DIST, DISTMediumHazard, DISTHighHazard
from fire_risk.models.DIST.providers.ahs import ahs_building_areas
from fire_risk.models.DIST.providers.iaff import response_time_distributions
from django.db.models import Avg, Max, Min, Q
from django.contrib.gis.geos import GEOSGeometry
from IPython.display import display
from firecares.utils import lenient_summation, dictfetchall
from firecares.tasks.update import (calculate_department_census_geom, calculate_story_distribution,
                                    calculate_structure_counts, update_performance_score, update_nfirs_counts,
                                    dist_model_for_hazard_level)
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 100)

def display_geom(geom):
    _map = folium.Map(location=[geom.centroid.y, geom.centroid.x],
                      tiles='Stamen Toner')
    _map.choropleth(geo_str=geom.geojson, line_weight=0, fill_opacity=0.2, fill_color='green')
    ll = geom.extent[1::-1]
    ur = geom.extent[3:1:-1]
    _map.fit_bounds([ll, ur])

    return _map

# Houston-specific
fd = FireDepartment.objects.get(id=84578)

#### Calculate quartiles for Houston based on similar departments

In [15]:
q = """SELECT "population_quartiles"."id",
       "population_quartiles"."created",
       "population_quartiles"."modified",
       "population_quartiles"."fdid",
       "population_quartiles"."name",
       "population_quartiles"."headquarters_address_id",
       "population_quartiles"."mail_address_id",
       "population_quartiles"."headquarters_phone",
       "population_quartiles"."headquarters_fax",
       "population_quartiles"."department_type",
       "population_quartiles"."organization_type",
       "population_quartiles"."website",
       "population_quartiles"."state",
       "population_quartiles"."region",
       "population_quartiles"."level",
       "population_quartiles"."dist_model_score",
       "population_quartiles"."risk_model_deaths",
       "population_quartiles"."risk_model_injuries",
       "population_quartiles"."risk_model_fires",
       "population_quartiles"."risk_model_fires_size0",
       "population_quartiles"."risk_model_fires_size0_percentage",
       "population_quartiles"."risk_model_fires_size1",
       "population_quartiles"."risk_model_fires_size1_percentage",
       "population_quartiles"."risk_model_fires_size2",
       "population_quartiles"."risk_model_fires_size2_percentage",
       "population_quartiles"."population",
       "population_quartiles"."population_class",
       "population_quartiles"."featured",
       "population_quartiles"."dist_model_score_quartile",
       "population_quartiles"."risk_model_deaths_quartile",
       "population_quartiles"."risk_model_injuries_quartile",
       "population_quartiles"."risk_model_fires_size0_quartile",
       "population_quartiles"."risk_model_fires_size1_quartile",
       "population_quartiles"."risk_model_fires_size2_quartile",
       "population_quartiles"."risk_model_fires_quartile",
       "population_quartiles"."risk_model_size1_percent_size2_percent_sum_quartile",
       "population_quartiles"."risk_model_size1_percent_size2_percent_sum",
       "population_quartiles"."risk_model_deaths_injuries_sum",
       "population_quartiles"."risk_model_deaths_injuries_sum_quartile",
       "population_quartiles"."residential_fires_avg_3_years",
       "population_quartiles"."residential_fires_avg_3_years_quartile",
       CASE
           WHEN ("population_quartiles"."dist_model_score" IS NOT NULL
                 AND "population_quartiles"."residential_fires_avg_3_years_quartile" IS NULL) THEN ntile(4) over (partition BY dist_model_score IS NOT NULL, residential_fires_avg_3_years_quartile, LEVEL IS NOT NULL
                                                                                                                  ORDER BY dist_model_score)
           ELSE NULL
       END AS "dist_model_residential_fires_quartile",
       CASE
           WHEN ("population_quartiles"."dist_model_score" IS NOT NULL
                 AND "population_quartiles"."risk_model_size1_percent_size2_percent_sum_quartile" = 4) THEN ntile(4) over (partition BY dist_model_score IS NOT NULL, risk_model_size1_percent_size2_percent_sum_quartile, LEVEL IS NOT NULL
                                                                                                                           ORDER BY dist_model_score)
           ELSE NULL
       END AS "dist_model_risk_model_greater_than_size_2_quartile",
       CASE
           WHEN ("population_quartiles"."dist_model_score" IS NOT NULL
                 AND "population_quartiles"."risk_model_deaths_injuries_sum_quartile" = 4) THEN ntile(4) over (partition BY dist_model_score IS NOT NULL, risk_model_deaths_injuries_sum_quartile, LEVEL IS NOT NULL
                                                                                                               ORDER BY dist_model_score)
           ELSE NULL
       END AS "dist_model_risk_model_deaths_injuries_quartile"
FROM "population_quartiles"
WHERE ("population_quartiles"."population_class" = 9
       AND "population_quartiles"."level" = 0)"""

pd.read_sql_query(q, connections['default'])

,id,created,modified,fdid,name,headquarters_address_id,mail_address_id,headquarters_phone,headquarters_fax,department_type,organization_type,website,state,region,level,dist_model_score,risk_model_deaths,risk_model_injuries,risk_model_fires,risk_model_fires_size0,risk_model_fires_size0_percentage,risk_model_fires_size1,risk_model_fires_size1_percentage,risk_model_fires_size2,risk_model_fires_size2_percentage,population,population_class,featured,dist_model_score_quartile,risk_model_deaths_quartile,risk_model_injuries_quartile,risk_model_fires_size0_quartile,risk_model_fires_size1_quartile,risk_model_fires_size2_quartile,risk_model_fires_quartile,risk_model_size1_percent_size2_percent_sum_quartile,risk_model_size1_percent_size2_percent_sum,risk_model_deaths_injuries_sum,risk_model_deaths_injuries_sum_quartile,residential_fires_avg_3_years,residential_fires_avg_3_years_quartile,dist_model_residential_fires_quartile,dist_model_risk_model_greater_than_size_2_quartile,dist_model_risk_model_deaths_injuries_quartile


#### Quartile view

In [14]:
with connections['default'].cursor() as c:
    c.execute("refresh materialized view population_quartiles;")

In [13]:
q = """SELECT
            (SELECT COALESCE(rm.risk_model_fires_size1_percentage,0)+COALESCE(rm.risk_model_fires_size2_percentage,0)) AS "risk_model_size1_percent_size2_percent_sum",
            (SELECT COALESCE(rm.risk_model_deaths,0)+COALESCE(rm.risk_model_injuries,0)) AS "risk_model_deaths_injuries_sum",
            fd."id",
            fd."created",
            fd."modified",
            fd."fdid",
            fd."name",
            fd."headquarters_address_id",
            fd."mail_address_id",
            fd."headquarters_phone",
            fd."headquarters_fax",
            fd."department_type",
            fd."organization_type",
            fd."website",
            fd."state",
            fd."region",
            fd."geom",
            rm."dist_model_score",
            rm."risk_model_deaths",
            rm."risk_model_injuries",
            rm."risk_model_fires",
            rm."risk_model_fires_size0",
            rm."risk_model_fires_size0_percentage",
            rm."risk_model_fires_size1",
            rm."risk_model_fires_size1_percentage",
            rm."risk_model_fires_size2",
            rm."risk_model_fires_size2_percentage",
            fd."population",
            fd."population_class",
            fd."featured",
            nfirs.avg_fires as "residential_fires_avg_3_years",
            rm."level",
            CASE WHEN (rm."risk_model_fires_size1_percentage" IS NOT NULL OR rm."risk_model_fires_size2_percentage" IS NOT NULL) THEN ntile(4) over (partition by COALESCE(rm.risk_model_fires_size1_percentage,0)+COALESCE(rm.risk_model_fires_size2_percentage,0) != 0, fd.population_class, rm.level order by COALESCE(rm.risk_model_fires_size1_percentage,0)+COALESCE(rm.risk_model_fires_size2_percentage,0)) ELSE NULL END AS "risk_model_size1_percent_size2_percent_sum_quartile",
            CASE WHEN (rm."risk_model_deaths" IS NOT NULL OR rm."risk_model_injuries" IS NOT NULL) THEN ntile(4) over (partition by COALESCE(rm.risk_model_deaths,0)+COALESCE(rm.risk_model_injuries,0) != 0, fd.population_class, rm.level order by COALESCE(rm.risk_model_deaths,0)+COALESCE(rm.risk_model_injuries,0)) ELSE NULL END AS "risk_model_deaths_injuries_sum_quartile",
            CASE WHEN rm."dist_model_score" IS NOT NULL THEN ntile(4) over (partition by rm.dist_model_score is not null, fd.population_class, rm.level order by rm.dist_model_score) ELSE NULL END AS "dist_model_score_quartile",
            CASE WHEN rm."risk_model_deaths" IS NOT NULL THEN ntile(4) over (partition by rm.risk_model_deaths is not null, fd.population_class, rm.level order by rm.risk_model_deaths) ELSE NULL END AS "risk_model_deaths_quartile",
            CASE WHEN rm."risk_model_injuries" IS NOT NULL THEN ntile(4) over (partition by rm.risk_model_injuries is not null, fd.population_class, rm.level order by rm.risk_model_injuries) ELSE NULL END AS "risk_model_injuries_quartile",
            CASE WHEN rm."risk_model_fires_size0" IS NOT NULL THEN ntile(4) over (partition by rm.risk_model_fires_size0 is not null, fd.population_class, rm.level order by rm.risk_model_fires_size0) ELSE NULL END AS "risk_model_fires_size0_quartile",
            CASE WHEN rm."risk_model_fires_size1" IS NOT NULL THEN ntile(4) over (partition by rm.risk_model_fires_size1 is not null, fd.population_class, rm.level order by rm.risk_model_fires_size1) ELSE NULL END AS "risk_model_fires_size1_quartile",
            CASE WHEN rm."risk_model_fires_size2" IS NOT NULL THEN ntile(4) over (partition by rm.risk_model_fires_size2 is not null, fd.population_class, rm.level order by rm.risk_model_fires_size2) ELSE NULL END AS "risk_model_fires_size2_quartile",
            CASE WHEN rm."risk_model_fires" IS NOT NULL THEN ntile(4) over (partition by rm.risk_model_fires is not null, fd.population_class, rm.level order by rm.risk_model_fires) ELSE NULL END AS "risk_model_fires_quartile",
            CASE WHEN "nfirs"."avg_fires" IS NOT NULL THEN ntile(4) over (partition by avg_fires is not null, fd.population_class, rm.level order by avg_fires) ELSE NULL END AS "residential_fires_avg_3_years_quartile"

        FROM "firestation_firedepartment" fd
        INNER JOIN "firestation_firedepartmentriskmodels" rm ON
            rm.department_id = fd.id
        LEFT JOIN (
            SELECT fire_department_id, AVG(count) as avg_fires, level
            from firestation_nfirsstatistic
            WHERE year >= 2010 and metric='residential_structure_fires'
            GROUP BY fire_department_id, level) nfirs
        ON (fd.id=nfirs.fire_department_id and nfirs.level = rm.level)
        WHERE archived=False"""

pd.read_sql_query(q, connections['default'])
    

,risk_model_size1_percent_size2_percent_sum,risk_model_deaths_injuries_sum,id,created,modified,fdid,name,headquarters_address_id,mail_address_id,headquarters_phone,headquarters_fax,department_type,organization_type,website,state,region,geom,dist_model_score,risk_model_deaths,risk_model_injuries,risk_model_fires,risk_model_fires_size0,risk_model_fires_size0_percentage,risk_model_fires_size1,risk_model_fires_size1_percentage,risk_model_fires_size2,risk_model_fires_size2_percentage,population,population_class,featured,residential_fires_avg_3_years,level,risk_model_size1_percent_size2_percent_sum_quartile,risk_model_deaths_injuries_sum_quartile,dist_model_score_quartile,risk_model_deaths_quartile,risk_model_injuries_quartile,risk_model_fires_size0_quartile,risk_model_fires_size1_quartile,risk_model_fires_size2_quartile,risk_model_fires_quartile,residential_fires_avg_3_years_quartile


## Full-cycle statistics import/calcuations

### At a minimum-we need to import the department and ALL related departments to get an accurate score

- Clear out FireDepartmentRiskModels, NFIRSStatistic
- Refresh `population_quartiles` materialized view (to ensure that we're starting w/ a clean slate)
- Run `update_nfirs` django command to pull NFIRS stats into FireCARES per department
    - Creates/updated NFIRSStatistic records
    - Depends on NFIRS.buildingfires, FC FireDepartment
- Run `calc-department-owned-tracts` django command to calculate the census-tract based department jurisdiction
    - Updates `FireDepartment.owned_tract_geom` on FC FireDepartment
    - Depends on NFIRS.census_block_groups_2010, NIST.tract_years, FC FireDepartment
- Run `calc-structure-counts` django command to determine the structure counts over a department's census-tract-based jurisdiction
    - Updates `FireDepartmentRiskModels.structure_count` on FC FireDepartmentRiskModels
    - Depends on NFIRS.parcel_risk_category_local, FC FireDepartment.owned_tracts_geom
- Run `calc-story-distribution` (not yet completed) to calculate the story distribution for a department's census-tract-based jurisdiction
    - Updates `FireDepartmentRiskModels.floor_count_coefficients`
    - Depends on NFIRS.LUSE_swg - copied from parcels.LUSE_swg, NFIRS.parcel_stories - subset of parcels.parcels, FC FireDepartment.owned_tracts_geom
- Run `import-predictions` to import predictions from Stanley's prediction model
    - Depends on predictions.csv file, FC FireDepartment
    - Recalculates the DIST score for EACH hazard level for FC FireDepartment
- Re-compute `population_quartiles` materialized view based on the imported information
    - Depends on FC Firedepartment, FC FireDepartmentRiskModels, FC NFIRSStatistic
- Run `load-dist-scores` to perform the DIST score calculation per department
    - Depends on NFIRS.buildingfires, NFIRS.incidentaddress, NFIRS.parcel_risk_category_local - copy of parcels.parcels w/ subset of columns + parcel risk level

In [ ]:
# Clear out FireDepartmentRiskModels since all of this information is derived
FireDepartmentRiskModels.objects.all().delete()
NFIRSStatistic.objects.all().delete()

with connections['default'].cursor() as c:
    c.execute("refresh materialized view population_quartiles;")

assert PopulationClassQuartile.objects.count() == 0

In [3]:
similar_departments = list(FireDepartment.objects.filter(id__in=fd.similar_departments.values_list('id', flat=True))) + [fd]

In [121]:
for f in similar_departments:
    update_nfirs_counts.delay(f.id)

#### Similar departments to Houston...

In [4]:
df = pd.DataFrame(map(lambda x: (x.id, x.name, x.population, x.region), similar_departments), columns=['id', 'name', 'population', 'region'])
df.sort('population', ascending=False)

,id,name,population,region
12,87255,Los Angeles County Fire Department,9818605,West
8,81472,Fire Department City of New York (FDNY),8175133,Northeast
9,83665,Harris County Fire & Emergency Services Depart...,4092459,South
13,87256,Los Angeles Fire Department,3792621,West
17,90354,North County Fire Protection District,3095313,West
18,91105,Orange County Fire Authority,3010232,West
2,77379,Chicago Fire Department,2695598,Midwest
14,88539,Miami-Dade Fire Rescue Department,2542139,South
23,93509,Riverside County Fire Department,2189641,West
27,84578,Houston Fire Department,2099451,South


In [122]:
df = pd.read_sql_query("select * from firestation_nfirsstatistic;", connections['default'])
department_count = len(df['fire_department_id'].unique())
year_count = len(df['year'].unique())
metric_type_count = len(df['metric'].unique())
# Should have 5 risk levels (low, medium, high, unknown, all) for each metrics for every year for each department
assert len(df) == department_count * 5 * metric_type_count * year_count

#### Now that we have NFIRS statistics (which is a dependency of the quartile calculations), we can continue

In [ ]:
# NOTE: this takes some time...
for f in similar_departments:
    calculate_department_census_geom.delay(f.id)
fd.refresh_from_db()
display_geom(fd.owned_tracts_geom)

In [ ]:
# NOTE: this takes some time...
from django.core.management import call_command

dept_ids = map(lambda x: str(x.id), similar_departments)
call_command('import-predictions', '../predictions.2015.csv', '--ids', *dept_ids)  # Also updates the performance score

In [130]:
with connections['default'].cursor() as c:
    c.execute('refresh materialized view population_quartiles;')
    
PopulationClassQuartile.objects.count()

6115

In [131]:
# NOTE: this takes a LONG time... (like hours)
for f in similar_departments:
    calculate_structure_counts.delay(f.id)

In [ ]:
for f in similar_departments:
    update_performance_score.delay(f.id)

In [7]:
q = """SELECT *
    FROM crosstab(
      'select COALESCE(y.risk_category, ''N/A'') as risk_category, fire_sprd, count(*)
        FROM buildingfires a left join (
          SELECT state,
            fdid,
            inc_date,
            inc_no,
            exp_no,
            geom,
            x.parcel_id,
            x.risk_category
          FROM (select * from incidentaddress a
             left join parcel_risk_category_local b using (parcel_id)
             ) AS x
        ) AS y using (state, inc_date, exp_no, fdid, inc_no)
    where a.state='%(state)s' and a.fdid='%(fdid)s' and prop_use in (''419'',''429'',''439'',''449'',''459'',''460'',''462'',''464'',''400'')
        and fire_sprd is not null and fire_sprd != ''''
    group by risk_category, fire_sprd
    order by risk_category, fire_sprd ASC')
    AS ct(risk_category text, "object_of_origin" bigint, "room_of_origin" bigint, "floor_of_origin" bigint, "building_of_origin" bigint, "beyond" bigint);"""

with connections['nfirs'].cursor() as c:
    c.execute(q, {'state': fd.state, 'fdid': fd.fdid})
    results = dictfetchall(c)

all_counts = dict(object_of_origin=0,
                  room_of_origin=0,
                  floor_of_origin=0,
                  building_of_origin=0,
                  beyond=0)

risk_mapping = {'Low': 1, 'Medium': 2, 'High': 4, 'N/A': 5}

print 'ALL RESULTS'
df = pd.DataFrame(results)
display(df)

for result in results:
    print result.get('risk_category')
    dist_model = dist_model_for_hazard_level(result.get('risk_category'))
    
    counts = dict(object_of_origin=result.get('object_of_origin', 0),
                  room_of_origin=result.get('room_of_origin', 0),
                  floor_of_origin=result.get('floor_of_origin', 0),
                  building_of_origin=result.get('building_of_origin', 0),
                  beyond=result.get('beyond', 0))
    
    display(counts)

    # add current risk category to the all risk category
    for key, value in counts.items():
        all_counts[key] += value
        
    ahs_building_size = ahs_building_areas(fd.fdid, fd.state)

    if ahs_building_size is not None:
        counts['building_area_draw'] = ahs_building_size

    response_times = response_time_distributions.get('{0}-{1}'.format(fd.fdid, fd.state))

    if response_times:
        counts['arrival_time_draw'] = LogNormalDraw(*response_times, multiplier=60)
        
    dist = dist_model(floor_extent=False, **counts)
    print 'SCORE: {}'.format(dist.gibbs_sample())

    
dist_model = dist_model_for_hazard_level('All')

if ahs_building_size is not None:
    all_counts['building_area_draw'] = ahs_building_size
    
if response_times:
    all_counts['arrival_time_draw'] = LogNormalDraw(*response_times, multiplier=60)
    
dist = dist_model(floor_extent=False, **all_counts)
    
display(all_counts)
print 'SCORE: {}'.format(dist.gibbs_sample())

ALL RESULTS


,beyond,building_of_origin,floor_of_origin,object_of_origin,risk_category,room_of_origin
0,9,60,20,70,High,97
1,151,1347,228,659,Low,1142
2,91,785,243,646,Medium,1171
3,215,1629,362,570,N/A,1577


High


{'beyond': 9L,
 'building_of_origin': 60L,
 'floor_of_origin': 20L,
 'object_of_origin': 70L,
 'room_of_origin': 97L}

SCORE: 1.0
Low


...

SCORE: 8.0
N/A


{'beyond': 215L,
 'building_of_origin': 1629L,
 'floor_of_origin': 362L,
 'object_of_origin': 570L,
 'room_of_origin': 1577L}

SCORE: 19.0


{'beyond': 466L,
 'building_area_draw':        h01_899  h01_990  h01_991  h01_992  h01_993  h01_994  h01_995  h01_996  \
 value     48.3    217.7    302.8    431.7    423.2    322.9    190.5    158.6   
 
        h01_997  h01_998  total  
 value     70.1    226.2   2392  ,
 'building_of_origin': 3821L,
 'floor_of_origin': 853L,
 'object_of_origin': 1945L,
 'room_of_origin': 3987L}

SCORE: 17.0


In [2]:
update_performance_score.delay(fd.id)

/webapps/firecares/local/lib/python2.7/site-packages/numpy/core/numeric.py:294: FutureWarning: in the future, full((10000, 4), -1000) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)



updating fdid: 84578 - High risk level from: 1.0 to 1.0.
updating fdid: 84578 - Low risk level from: 19.0 to 19.0.
updating fdid: 84578 - Medium risk level from: 8.0 to 8.0.
updating fdid: 84578 - Unknown risk level from: 21.0 to 21.0.
updating fdid: 84578 - All risk level from: 30.0 to 17.0.


<EagerResult: a3aa8655-4bc6-4382-821c-9330f62318e0>